## IMPORTANT

Ensure you have created your virtual environment and installed the requirements. 

In [1]:
from IPython.display import display, Image, clear_output
import pandas as pd
import random 
import re 
import numpy as np
import music21 as m21
import time
import datetime
import nltk 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

# 200806695 MSC Project

## An intelligent tutoring system (ITS) for music theory teaching

In [2]:
# data = './q_folders/stave/labels_treble.csv'
# data = pd.read_csv(data)
# data.head()


In [3]:
# data = './q_folders/transpose/bb/labels.csv'
# # data = './q_folders/stave/labels_treble.csv'
# data = pd.read_csv(data)
# data.head()


In [4]:
# #random number between 0 and the length of the table
# rand_ex = random.randint(0,len(data)-1)

# file, label = data.iloc[rand_ex, 1], data.iloc[rand_ex, 2]
# print ("The file is called {} and the answer is {}" .format(file, label))
# display(Image(filename='./q_folders/transpose/bb/' + file))

In [5]:
# def identificationOpen(data): 
#     qs = pd.read_csv(data)
    
#     score = 0
#     total = 0
#     combo = 0 
    
#     while True:
        
#         clear_output(wait=True)
        
#         print ('Answer with letters A - G, type stop to end the game')
#         print (score, '/', total)
#         print ('combo:', combo)
                
#         #get q at random
#         ex = random.randint(0,len(qs)-1)
#         file, label = qs['pngfile'][ex], qs['label'][ex]  
        
#         print ('What note is this? \n')

#         display(Image(filename='./q_folders/stave/' + file))

#         answer = input('')
    
#         #noramlise text
#         answer = answer.upper()
#         answer = answer.strip()

#         if answer == label:
#             print ("Well done!")
#             score += 1
#             total += 1
#             combo += 1

#             continue

#         if answer == 'STOP':
#             if total == 0:
#                 return ('done')
#             else:
#                 print ('You score', round(((score/total)*100), 2), '%')
#                 return score
#         else:
#             print("incorrect, try again")
#             total += 1
#             combo = 0

In [6]:
#get bank of questions
data = './q_folders/stave/labels_treble.csv'

#ask q's
# identificationOpen(data)

In [7]:
def scoreCard(qs, student_no=None, init=False, finished=None, question=None, correct=None, update_probs=True):
    
    if init == True:

        #make new columns, initialise with 1 to avoid division by zero error
        qs['#_attempts'] = 1
        qs['#_correct'] = 1

        #add probabilities to dataframe
        qs['probability'] = qs['#_attempts'] / qs['#_correct']
        qs['probability'] = qs['probability'] / qs['probability'].sum()
        
    if correct == True:

        #correct
        qs.loc[qs.index.values == question, '#_attempts'] += 1
        qs.loc[qs.index.values == question, '#_correct'] += 1
        
    if update_probs and correct == True:

        #subtract probability mass from question
        qs.loc[qs.index.values == question, 'probability'] = (qs.loc[qs.index.values == question, 'probability'] / 2)
        
    if correct == False:

        #incorrect
        qs.loc[qs.index.values == question, '#_attempts'] += 1
        
    if update_probs and correct == False:

        #add probability mass to question  
        qs.loc[qs.index.values == question, 'probability'] = (qs.loc[qs.index.values == question, 'probability'] * 2)
        
    #normalise probabilities
    qs['probability'] = qs['probability'] / qs['probability'].sum()
    probabilities = qs['probability'].to_numpy() #variable to be accessed by method

    if finished == True:
        
        #minus the 1 that we added earlier to avoid division by zero 
        qs['#_attempts'] -= 1
        qs['#_correct'] -= 1
        
        time_stamp = datetime.datetime.now().strftime('%c')
        time_stamp = str(time_stamp)
        time_stamp = time_stamp.replace(':', '-')
        
        file_name = '{}_{}_.csv'.format(student_no, time_stamp)
        qs.to_csv(file_name, index=False)
        return 

In [8]:
def generateQ(q_index, combo, score, total, probabilities, ex=None, interval=False): 

    if interval == True:
        ex = ex + np.random.choice(np.arange(-1,2), p=[1/2,0,1/2])
#         ex = ex + generateQ(q_index, probabilities, init=False, interval=True)
        
#     #ask again if they got last q wrong
        return ex

    else: 
        #draw from probabilities
#         ex = generateQ(q_index, probabilities, init=False, interval=False)
        ex = np.random.choice(q_index, p=probabilities)

    return ex


In [9]:
def answerCleanup(answer, regex=None, method=None):
            
    if method == 'generative_int':
        answer = answer.strip()
        return answer
    
    if method == 'generative_note' or method == 'notes_advanced':
        answer = answer.replace(' ', '')
        answer = answer.upper()
        
        regex = r"[a-gA-G 0-9\#\b]"
        
    if regex: 

        p = re.compile(regex)
        m = p.match(answer)

        if m:
            return answer
        else:
            return 'invalid'
    
    else:
        return answer

In [12]:
def generative_int(down=True, qs=None, ex=None):
    
    interval = qs['label'][ex]
    answer = qs['label'][ex]
    
    pitches = ['F2','G2','A2', 'B2',
            'C3', 'D3', 'E3', 'F3', 'G3', 'A3', 'B3',
            'C4', 'D4', 'E5', 'F5', 'G5']

    p1 = m21.pitch.Pitch(random.choice(pitches)) #pitch1 
    n1 = m21.note.Note(p1) #create note1 from pitch1
    n1.duration.quarterLength = 2
    if down == True:
        p2 = random.choice([p1.transpose('-' + interval), p1.transpose(interval)])
#         p2 = p1.transpose('-' + q) #transpose down
    else:
        p2 = p1.transpose(interval) #transpose up

    #p2.nameWithOctave #if you want to check
    n2 = m21.note.Note(p2) #create note2 from transposed pitch1
    n2.duration.quarterLength = 2

    s = m21.stream.Stream() #create stream
    s.append(n1) #add notes to stream
    s.append(n2)

    return s, answer

In [13]:
def generative_note(ex=None, qs=None, data=None):
    
    #note = qs['note'][ex] #needs to be note variable for m21
    
    note_values = ['whole', 'half', 'quarter', 'eighth']
    
    clef = qs['clef'][ex]
    
    if clef == 'TrebleClef':
        clef = m21.clef.TrebleClef()
    if clef == 'BassClef':
        clef = m21.clef.BassClef()
        
    answer = qs['label'][ex]
    
#     ks = m21.key.KeySignature(int(qs['ks'][ex]))
    
#     alteredPitches = [str(p) for p in ks.alteredPitches]    

    s = m21.stream.Measure([clef])
#     s.insert(0, ks)
    ts1 = m21.meter.TimeSignature('4/4')
    ts1.symbol = 'None'
    s.append(ts1)
    
    p1 = m21.pitch.Pitch(qs['pitches'][ex]) #pitch1 
    
#     if p1.accidental:
#         p1.accidental = None
        
#     if note[0:2] in alteredPitches and p1.accidental == True:
#          p1.accidental = None
#     else:
#         pass
    
    n1 = m21.note.Note(p1, type=random.choice(note_values)) #create note1 from pitch1
    s.append(n1) #add notes to stream
    
#     if transpose:
#         s.transpose(transpose)
        
    return s, answer

In [14]:
def notes_advanced(qs=None, ex=None, transpose=None):
    
    tonic = qs['tonic'][ex]
    clef = qs['clef'][ex]
    start_range = qs['start'][ex]
    end_range = qs['end'][ex]
    no_notes = qs['no_notes'][ex]
    
    #clef
    if clef == 'TrebleClef':
        clef = m21.clef.TrebleClef()
    if clef == 'BassClef':
        clef = m21.clef.BassClef()
        
    s = m21.stream.Measure([clef])
    #time sig
    ts1 = m21.meter.TimeSignature('4/4')
    ts1.symbol = 'None'
    s.append(ts1)
    #key
    k = m21.key.Key(tonic)
    s.append(k) #add key sig 
    
    #scale
    sc1 = m21.scale.MajorScale(tonic)
    
#     if transpose:
#         sc1 = sc1.transpose(transpose) #transpose notes
#     else:
#         pass
        
    pitches = [str(p) for p in sc1.getPitches(start_range, end_range)]
    notes = random.sample(pitches, no_notes)
#     notes = ' '.join([str(note) for note in notes])

    if transpose:
        k = k.transpose(transpose) #if you want to change the key signature
        # k.transpose(transpose) #if you want to change the notes but not the key, could be a param

    if transpose:
        sc1 = sc1.transpose(transpose) #transpose scale
    else:
        pass
    
#     print (notes)
    #transpose notes
    
    notes_list = []
    
    for note in notes:
        p = m21.pitch.Pitch(note)
        n = m21.note.Note(p, type='quarter') #create note1 from pitch1
        s.append(n)
        if transpose:
            p = p.transpose(transpose)
        else:
            pass
        notes_list.append(str(p.name).replace('-', 'B'))

        
    answer = notes_list

    return s, answer

In [15]:
# its()

In [16]:
def game(data, instructions, prompt, method, student_no, min_q=5, pass_threshold=0.9, down=False, 
         blind=True, difficulty=None, transpose=None):
    
    times = []
    
    #convert csv of qs to dataframe
    qs = pd.read_csv(data + 'labels.csv')
    
    #slice data to q's according to difficulty
    if difficulty:
        qs = qs.loc[qs['difficulty'] == difficulty].reset_index(drop=True)
    else:
        pass
    
    #initialise score card based on csv of questions, get init probabilities (random)
    scoreCard(qs, init=True)
    probabilities = qs['probability']
    
    #the index will be used by generateQ to choose questions 
    q_index = qs.index.values
    
    #initialise local variables to track scores and give instant feedback
    score = 0
    total = 0
    combo = 0 
    
    #feedback is used to help guide the student
    feedback = "Okay, here we go"
    
    finished = False
    
    previous_correct = True
    
    #first loop: pick a question using lower level strategies
    while True:

        #ask again if previous answer incorrect
        if total == 0:
            ex = generateQ(q_index=q_index, score=score, combo=combo,total=total,
                        interval=False, probabilities=probabilities)
            
        #restrict to interval in case of ordinal q's
        elif score/total < 0.8:
            ex = generateQ(ex=ex, q_index=q_index, score=score, combo=combo,total=total,
                        interval=True, probabilities=probabilities)
            
        #draw from probabilities
        else:
            ex = generateQ(q_index=q_index, score=score, combo=combo, total=total,
                        interval=False, probabilities=probabilities)
#             continue

        #second loop: present the question, get answer, update scores and probabilities
        while True:

            if total > 0 and total % min_q == 0 and score/total > pass_threshold:
                answer = input('''You\'re doing well, do you want to:
                1. end the game and try something new or
                2. continue playing?
                Type 1 or 2 )''')
                
                while True:

                    answer = answer[0]
                    answer = answerCleanup(answer, regex="[1-2]")
                    
                    if answer == 'invalid':
                        print ('invalid answer, type 1 or 2')
                        continue
                        
                    else:

                        if answer == '1':
                            finished = True
                            break
                        
                        if answer == '2':
                            break

#             print ('ex is {}'.format(ex))
#             print  (qs['label'])
            
#             label = qs['label'][ex] #correct answer 
            clear_output(wait=True)
            
            if finished == True: 
                scoreCard(qs, student_no, finished=True)
                return score/total
    
            print (feedback, '\n \n {} \n'.format(instructions)) #this could be a prompt?
            # and the prompt could be different things depending on how the game is going and the curricula

            print (score, '/', total)
            print ('combo:', combo)
                        
            #ASK QUESTION
            if method == 'generative_note':
                s, label = generative_note(qs=qs, ex=ex)
                s.show()
                
            if method == 'notes_advanced':
                s, label = notes_advanced(
                    ex=ex, qs=qs, transpose=transpose)
                s.show()
                
                label = ''.join(str(note).upper() for note in label)
                print (label)
                
            if method == 'generative_int':
                if previous_correct == False:
                    pass
                else:
                    s, label = generative_int(down=True, ex=ex, qs=qs)
                if blind == False:
                    s.show() #show music
                else:
                    pass
                sp = m21.midi.realtime.StreamPlayer(s) 
                sp.play()
                
            print (prompt) 
            #GET ANSWER
            start_time = time.time()
            answer = input('') #hash out to compute answer automatically 
            inputs.append(answer)

            if answer == 'stop': 
                finished = True
                continue

#                 if total == 0:
#                     return ('No questions answered')
#                 else:
#                     print ('You score', round(((score/total)*100), 2), '%')
#     #                 return score
#                     return scoreCard(qs, finished=True)
            
            answer = answerCleanup(answer, method=method) #unhash to test answer
        
            if answer == label:
                end_time = time.time()
                time_lapsed = end_time - start_time
                times.append(time_lapsed)
                
                score += 1
                total += 1
                combo += 1
                previous_correct = True

                if input('Well done! Press enter to continue.') == '':
                    pass
                if previous_correct == True:
                    scoreCard(qs, question=ex, correct=True, update_probs=True)
                    break
                
                #do not update probabilities if last question was incorrect
                if previous_correct == False:
                    scoreCard(qs, question=ex, correct=True, update_probs=False)
                    break
    
            if answer == 'invalid':
                if input('WARNING: Invalid answer.') == '':
                    pass
        
                continue

            else:
                total += 1
                combo = 0
                previous_correct = False
                scoreCard(qs, question=ex, correct=False, update_probs=True)
                if input('Incorrect - press enter to try again.') == '':
                    pass
                
                continue

In [17]:
def preProcess(word):
    porter = PorterStemmer()
    Lem = WordNetLemmatizer()

    word = Lem.lemmatize(word)
    word = porter.stem(word)
    return word

def preProcessList(lst):
    words = []
    for word in lst:
        word = preProcess(word)
        words.append(word)
        
    return words

In [18]:
#key words for different subjects (bag of words)
notation = ['notes', 'clef', 'reading', 'notation']
interval = ['interval', 'distance', 'major', 'minor']
transposition = ['transpose', 'trumpet', 'instruments', 'transposition']

#preProcess lists
notation = preProcessList(notation)
interval = preProcessList(interval)
transposition = preProcessList(transposition)


In [19]:
#identification method

method = 'generative_note'
instructions = 'Answer with letters A-G. Type stop to end the game'
regex = r"[a-gA-G 0-9\#\b]"
prompt = 'What pitch is displayed on the clef?'
data = '/q_folders/g1_treble_'



In [20]:
#interval method
method = 'generative_int'
instructions = 'Enter your answer with 2 chars only'

data = '/q_folders/interval_'
prompt = 'Which interval did you hear?'


In [21]:
def subjectSelect(greeting, student_no):

    while True:
        
        #parse input
        tokens = greeting.split()

        for token in tokens:
            token = preProcess(token)



            #subject select
            if token in notation:
                print ('Okay, would you like to train on the treble or bass clef?')

                method = 'generative_note'
                instructions = 'Answer with letters A-G. Type stop to end the game'
                regex = r"[a-gA-G 0-9\#\b]"
                prompt = 'What pitch is displayed on the clef?'

                answer = input()
                if answer == 'treble':
                    ('Okay, lets do notes on the treble clef')
                    data = './q_folders/g1_treble_' #treble
                    score = game(data, instructions, prompt, method, student_no,
                                min_q=5, pass_threshold=0.9) #start game

                if answer == 'bass':
                    ('Okay, lets do notes on the bass clef')
                    data = './q_folders/g1_bass_' #bass
                    score = game(data, instructions, prompt, method, student_no,
                                min_q=5, pass_threshold=0.9) #start game

                if score < 0.5:
                    print ('You should try something easier')
                if score > 0.5:
                    print ('You should try something harder')


    #             if answer == 'alto':
    #                 ('Okay, let\s do notes on the alto clef.')
    #                 identificationOpen('./q_folders/altoclef/')

            if token in interval:
                print ('Okay, let\s do interval training')

                #interval method
                method = 'generative_int'
                instructions = 'Enter your answer with 2 chars only'
                data = './q_folders/interval_'
                prompt = 'Which interval did you hear?'

                difficulty = 1

                while True:

                    next_instruction = game(data, instructions, prompt, method, student_no,
                                            min_q=5, pass_threshold=0.5, difficulty=difficulty) #start game

                    if next_instruction == 'level_up':
                        difficulty += 1
                        continue
                    if next_instruction == 'stop':
                        break 



            if token in transposition:

                print ('Okay, let\s do transposition')
                data = './q_folders/transposition_'
                prompt = 'What is the sounding pitch of this note for a Bb instrument?'
                instructions = 'Answer with letters A-G. Type stop to end the game'
                method = 'notes_advanced'
                transpose = 'm2'
                difficulty = 2
                return game(data, instructions, prompt, method, student_no, transpose='-M2',
                                             pass_threshold=0.5, difficulty=difficulty)

                if next_instruction == 'level_up':
                    data = './q_folders/transpose/eb/'
                    return game(data, instructions, prompt, 'identificationOpen')

        return 'invalid'


In [22]:
def its():
    
    global inputs
    inputs = []
    
    student_no = input('Please enter your username')
    inputs.append(student_no)
    
    while True:
        print ('Thanks, {}. What do you want to learn today?'.format(student_no))
        print ('Type stop to end the session')
        answer = input()
        inputs.append(answer)
        if answer == 'stop':
            print (inputs)
            break
        else:
            subjectSelect(answer, student_no=student_no)

In [23]:
its()

ZeroDivisionError: division by zero

_________